<h1 align=center><font size = 5>Neighborhoods comparison in Madrid and Barcelona</font></h1>

## Introduction/Business Problem

We will compare the two biggest cities in Spain: Madrid and Barcelona.
The Business problem we want to address is to discover the neighborhoods that are similar between the two cities related to the category and number of venues that they have.
We will cluster the neighborhoods using k-means and then will examine the clusters to find similarities between the two cities.
We will analyse what makes them more attractive to tourism.
The target audience are franchise operators that focus on tourism in these two cities and need to know where to open the franchise.


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Neighborhoods of Madrid

In [2]:
url = 'https://es.wikipedia.org/wiki/Anexo:Barrios_administrativos_de_Madrid'
html = requests.get(url).content
df_list = pd.read_html(html)
df_mad = df_list[-1]
df_mad.head()

,Distrito,Número,Nombre,Superficie (km²)[2]​,Imagen
0,Centro,11,Palacio,"1,471 km²",NaN
1,Centro,12,Embajadores,"1,032 km²",NaN
2,Centro,13,Cortes,"0,592 km²",NaN
3,Centro,14,Justicia,"0,742 km²",NaN
4,Centro,15,Universidad,"0,947 km²",NaN


In [54]:
df_mad2= df_mad['Nombre']

df_madrid = df_mad2.to_frame()
df_madrid = df_madrid.rename(columns={'Nombre': 'Neighborhood'})
df_madrid.head()

,Neighborhood
0,Palacio
1,Embajadores
2,Cortes
3,Justicia
4,Universidad


In [55]:
df_madrid['Neighborhood'].isnull().values.any()

False

### Barrios de Barcelona

In [56]:
url = 'https://es.wikipedia.org/wiki/Distritos_de_Barcelona'
html = requests.get(url).content
df_list = pd.read_html(html)
df_bcn = df_list[-2]
df_bcn = df_bcn.rename(columns={'Barrios (nº)': 'Neighborhood'})
df_bcn.head()

,Nº,Distrito,Imagen,Superficie km²[1]​,Población (2019)[2]​,Densidad hab/km²,Neighborhood,Regidor
0,1,Ciudad Vieja,NaN,411,103 429,"25 159,09","El Raval (1), Barrio Gótico (2), La Barcelonet...",Gala Pin Ferrando (Barcelona en Comú)
1,2,Ensanche,NaN,746,265 910,"35 625,67","El Fort Pienc (5), Sagrada Familia (6), Dreta ...",Agustí Colom Cabau (Barcelona en Comú)
2,3,Sants-Montjuïc,NaN,2268,184 091,811832,"Pueblo Seco (11), La Marina del Prat Vermell (...",Jaume Asens Llodrà (Barcelona en Comú)
3,4,Les Corts,NaN,602,81 974,"13 607,11","Les Corts (19), La Maternidad y San Ramón (20)...",Laura Pérez Castaño (Barcelona en Comú)
4,5,Sarriá-San Gervasio,NaN,1991,149 260,749711,"Vallvidrera, el Tibidabo i les Planes (22), Sa...",Albert Batlle (Units per Avançar)


In [57]:
df1=df_bcn1[0]+ ', '
for i in range(df_bcn1.shape[0]-1) :
      df1 = df1 + df_bcn1[i+1] + ', '    

with open("BcnBarrios.csv","w") as file:
    file.write(df1 + "\n")


In [58]:
df_bcn2 = pd.read_csv('BcnBarrios1.csv', encoding="latin-1")
df_barcelona = df_bcn2.T.reset_index()
df_barcelona.columns = ['Neighborhood']
df_barcelona.head()

,Neighborhood
0,El Raval
1,Barrio Gótico
2,La Barceloneta
3,Santa Caterina i la Ribera
4,El Fort Pienc


In [59]:
df_barcelona['Neighborhood'].isnull().values.any()

False

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.<br/>
There are no boroughs with an empty neighborhood.

#### Shape of the Dataframes

In [13]:
df_madrid.shape

(131, 1)

In [14]:
df_barcelona.shape

(77, 1)

#### Use geopy library to get the latitude and longitude values of Madrid Neighborhoods.

In [44]:
for i in range(len(df_madrid)) :
    print( df_madrid.iloc[i,0])

Palacio
Embajadores
Cortes
Justicia
Universidad
Sol
Imperial
Acacias
Chopera
Legazpi
Delicias
Palos de Moguer
Atocha
Pacífico
Adelfas
Estrella
Ibiza
Jerónimos
Niño Jesús
Recoletos
Goya
Fuente del Berro
La Guindalera
Lista
Castellana
El Viso
Prosperidad
Ciudad Jardín
Hispanoamérica
Nueva España
Castilla
Bellas Vistas
Cuatro Caminos
Castillejos
Almenara
Valdeacederas
Berruguete
Gaztambide
Arapiles
Trafalgar
Almagro
Ríos Rosas
Vallehermoso
El Pardo
Fuentelarreina
Peñagrande
El Pilar
La Paz
Valverde
Mirasierra
El Goloso
Casa de Campo
Argüelles
Ciudad Universitaria
Valdezarza
Valdemarín
El Plantío
Aravaca
Los Cármenes
Puerta del Ángel
Lucero
Aluche
Campamento
Cuatro Vientos
Las Águilas
Comillas
Opañel
San Isidro
Vista Alegre
Puerta Bonita
Buenavista
Abrantes
Orcasitas
Orcasur
San Fermín
Almendrales
Moscardó
Zofío
Pradolongo
Entrevías
San Diego
Palomeras Bajas
Palomeras Sureste
Portazgo
Numancia
Pavones
Horcajo
Marroquina
Media Legua
Fontarrón
Vinateros
Ventas
Pueblo Nuevo
Quintana
Concepció

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [60]:
data = []
df_madll = pd.DataFrame(data, columns=['Neighborhood', 'Latitude', 'Longitude'])

for i in range(len(df_madrid)) :
    address = df_madrid.iloc[i,0]+', Madrid, Spain'
    try:
        geolocator = Nominatim(user_agent="ny_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
    
        df_madll = df_madll.append({'Neighborhood':df_madrid.iloc[i,0],'Latitude':latitude,'Longitude':longitude}, ignore_index=True)

                            
        #print('{}, {}, {}'.format(df_madrid.iloc[i,0], latitude, longitude))
    
    except: 
        print( df_madrid.iloc[i,0])

Los Cármenes
Valderrivas
Casco Histórico de Barajas


In [94]:
#df_madll = pd.read_csv('MadridLatLong.csv', encoding='latin-1')
#df_madll = pd.read_csv('Mad1.txt', encoding='iso-8859-1')

In [61]:
df_madll.head()

,Neighborhood,Latitude,Longitude
0,Palacio,40.415129,-3.715618
1,Embajadores,40.409681,-3.701644
2,Cortes,40.414779,-3.697584
3,Justicia,40.423957,-3.695747
4,Universidad,40.425409,-3.705989


#### Use geopy library to get the latitude and longitude values of Barcelona Neighborhoods.

In [62]:
data = []
df_bcnll = pd.DataFrame(data, columns=['Neighborhood', 'Latitude', 'Longitude'])

for i in range(len(df_barcelona)) :
    address = df_barcelona.iloc[i,0]+', Barcelona, Spain'
    try:
        geolocator = Nominatim(user_agent="ny_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
    
        df_bcnll = df_bcnll.append({'Neighborhood':df_barcelona.iloc[i,0],'Latitude':latitude,'Longitude':longitude}, ignore_index=True)

                            
        #print('{}, {}, {}'.format(df_barcelona.iloc[i,0], latitude, longitude))
    
    except: 
        print( df_barcelona.iloc[i,0])

La Antigua Izquierda del Ensanche
La Nueva Izquierda del Ensanche
Pueblo Seco
Vallcarca y los Penitentes
Villa de Gracia
El Valle de Hebrón
El Campo del Arpa del Clot
Pueblo Nuevo
San Martín de Provensals


In [63]:
df_bcnll.head()

,Neighborhood,Latitude,Longitude
0,El Raval,41.379518,2.168368
1,Barrio Gótico,41.383395,2.176912
2,La Barceloneta,41.380653,2.189927
3,Santa Caterina i la Ribera,41.386650,2.184194
4,El Fort Pienc,41.395925,2.182325


#### Create a map of Madrid with neighborhoods superimposed on top.

In [20]:
address = 'Madrid, Spain'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Madrid using latitude and longitude values
map_madrid = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df_madll['Latitude'], df_madll['Longitude'], df_madll['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madrid)  
    
map_madrid

Discard outer points?

#### Create a map of Barcelona with neighborhoods superimposed on top.

In [22]:
address = 'Barcelona, Spain'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Barcelona using latitude and longitude values
map_barcelona = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df_bcnll['Latitude'], df_bcnll['Longitude'], df_bcnll['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_barcelona)  
    
map_barcelona

#### Let's cluster the neighborhoods of Madrid and Barcelona

We start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [23]:
CLIENT_ID = 'INTDEIQ04DX555YN1MTDX44V5CAPBR5ABIA5PBTZO0FDOZJI' # your Foursquare ID
CLIENT_SECRET = 'WFA4IQIH4SU5ZI3TLVVRBEEO1UUQ3YBHBPYNRPTVUN1RMRW1' # your Foursquare Secret
VERSION = '20180604'
##LIMIT = 30



#### Let's explore the first Madrid neighborhood in our dataframe.

Get the neighborhood's name.

In [24]:
df_madll.loc[0, 'Neighborhood']

'Palacio'

In [25]:
neighborhood_latitude = df_madll.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_madll.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_madll.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Palacio are 40.41512925, -3.7156179983990922.


#### Now, let's get the top 100 venues that are in this neighborhood within a radius of 500 meters.

In [26]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius
 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=INTDEIQ04DX555YN1MTDX44V5CAPBR5ABIA5PBTZO0FDOZJI&client_secret=WFA4IQIH4SU5ZI3TLVVRBEEO1UUQ3YBHBPYNRPTVUN1RMRW1&v=20180604&ll=40.41512925,-3.7156179983990922&radius=500&limit=100'

In [27]:
results = requests.get(url).json()
#results

{'meta': {'code': 200, 'requestId': '5ee20cb4237de169927dd1ea'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Palacio',
  'headerFullLocation': 'Palacio, Madrid',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 48,
  'suggestedBounds': {'ne': {'lat': 40.4196292545, 'lng': -3.7097186073804784},
   'sw': {'lat': 40.4106292455, 'lng': -3.721517389417706}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4adcda38f964a520523c21e3',
       'name': 'Santa Iglesia Catedral de Santa María la Real de la Almudena (Catedral de la Almudena)',
       'location': {'address': 'C. Bailén, 8-10',
        'lat': 40.41576693264202,
        'lng': -3.7145161628723145,
        'label

In [28]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean json and structure it into a *pandas* dataframe.

In [29]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\Castro\.conda\envs\myenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Santa Iglesia Catedral de Santa María la Real ...,Church,40.415767,-3.714516
1,Plaza de La Almudena,Plaza,40.416320,-3.713777
2,Cervecería La Mayor,Beer Bar,40.415218,-3.712194
3,Taberna Rayuela,Tapas Restaurant,40.413179,-3.713496
4,Palacio Real de Madrid,Palace,40.417940,-3.714259


In [30]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

48 venues were returned by Foursquare.


## Explore Neighborhoods in Madrid

#### Let's create a function to repeat the same process to all the neighborhoods 

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=250):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

##### Code to run the above function on each neighborhood and create a new dataframe called *dmadrid_venues*.

In [36]:
dmadrid_venues = getNearbyVenues(names=df_madll['Neighborhood'],
                                   latitudes=df_madll['Latitude'],
                                   longitudes=df_madll['Longitude']
                                  )

Palacio
Embajadores
Cortes
Justicia
Universidad
Sol
Imperial
Acacias
Chopera
Legazpi
Delicias
Palos de Moguer
Atocha
Pacífico
Adelfas
Estrella
Ibiza
Jerónimos
Niño Jesús
Recoletos
Goya
Fuente del Berro
La Guindalera
Lista
Castellana
El Viso
Prosperidad
Ciudad Jardín
Hispanoamérica
Nueva España
Castilla
Bellas Vistas
Cuatro Caminos
Castillejos
Almenara
Valdeacederas
Berruguete
Gaztambide
Arapiles
Trafalgar
Almagro
Ríos Rosas
Vallehermoso
El Pardo
Fuentelarreina
Peñagrande
El Pilar
La Paz
Valverde
Mirasierra
El Goloso
Casa de Campo
Argüelles
Ciudad Universitaria
Valdezarza
Valdemarín
El Plantío
Aravaca
Puerta del Ángel
Lucero
Aluche
Campamento
Cuatro Vientos
Las Águilas
Comillas
Opañel
San Isidro
Vista Alegre
Puerta Bonita
Buenavista
Abrantes
Orcasitas
Orcasur
San Fermín
Almendrales
Moscardó
Zofío
Pradolongo
Entrevías
San Diego
Palomeras Bajas
Palomeras Sureste
Portazgo
Numancia
Pavones
Horcajo
Marroquina
Media Legua
Fontarrón
Vinateros
Ventas
Pueblo Nuevo
Quintana
Concepción
San Pascual

In [37]:
print('There are {} uniques categories.'.format(len(dmadrid_venues['Venue Category'].unique())))

There are 200 uniques categories.


In [38]:
dmadrid_venues.to_csv("madrid_venues.csv")

#### Analyze Each Neighborhood

In [64]:
# one hot encoding
dmadrid_onehot = pd.get_dummies(dmadrid_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dmadrid_onehot['Neighborhood'] = dmadrid_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dmadrid_onehot.columns[-1]] + list(dmadrid_onehot.columns[:-1])
dmadrid_onehot = dmadrid_onehot[fixed_columns]

dmadrid_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Embassy / Consulate,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Health & Beauty Service,Health Food Store,Historic Site,Hobby Shop,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Library,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Racetrack,Record Shop,Rental Car Location,Restaurant,Russian Restaurant,Salad Place,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track Stadium,Trade School,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Palacio,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Palacio,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Palacio,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### Group by neighborhood

In [65]:
dmadrid_grouped = dmadrid_onehot.groupby('Neighborhood').mean().reset_index()
dmadrid_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Embassy / Consulate,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Health & Beauty Service,Health Food Store,Historic Site,Hobby Shop,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Library,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Racetrack,Record Shop,Rental Car Location,Restaurant,Russian Restaurant,Salad Place,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track Stadium,Trade School,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,Abrantes,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

##### Let's print each neighborhood along with the top 5 most common venues

In [66]:
num_top_venues = 5

for hood in dmadrid_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dmadrid_grouped[dmadrid_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Abrantes----
                  venue  freq
0           Pizza Place  0.25
1  Fast Food Restaurant  0.25
2         Metro Station  0.25
3                Bakery  0.25
4           Yoga Studio  0.00


----Acacias----
                  venue  freq
0    Spanish Restaurant  0.09
1                  Café  0.09
2  Gym / Fitness Center  0.09
3           Supermarket  0.09
4      Tapas Restaurant  0.09


----Adelfas----
                venue  freq
0    Tapas Restaurant  0.18
1                 Spa  0.09
2         Coffee Shop  0.09
3  Spanish Restaurant  0.09
4       Grocery Store  0.09


----Alameda de Osuna----
                venue  freq
0    Tapas Restaurant   0.2
1          Smoke Shop   0.1
2          Hobby Shop   0.1
3         Pizza Place   0.1
4  Italian Restaurant   0.1


----Almagro----
                venue  freq
0  Spanish Restaurant  0.19
1          Restaurant  0.07
2           Gastropub  0.07
3         Salad Place  0.07
4                Café  0.04


----Almenara----
                  v

##### Let's put it into a *pandas* dataframe

In [67]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [76]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dmadrid_grouped['Neighborhood']

for ind in np.arange(dmadrid_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dmadrid_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abrantes,Bakery,Metro Station,Pizza Place,Fast Food Restaurant,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Fountain
1,Acacias,Gym / Fitness Center,Supermarket,Tapas Restaurant,Café,Spanish Restaurant,Indie Movie Theater,Sushi Restaurant,Bookstore,Bar,Gym
2,Adelfas,Tapas Restaurant,Spa,Hotel,Gym,Supermarket,Coffee Shop,Asian Restaurant,Grocery Store,Spanish Restaurant,Breakfast Spot
3,Alameda de Osuna,Tapas Restaurant,Pizza Place,Metro Station,Italian Restaurant,Fried Chicken Joint,Smoke Shop,Hobby Shop,Restaurant,Bakery,French Restaurant
4,Almagro,Spanish Restaurant,Gastropub,Salad Place,Restaurant,Plaza,Pub,Café,Breakfast Spot,Cocktail Bar,Coffee Shop


###  Cluster Madrid Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [77]:
# set number of clusters
kclusters = 5

dmadrid_grouped_clustering = dmadrid_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dmadrid_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [78]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dmadrid_merged = df_madll

# merge madrid_grouped with madrid_data to add latitude/longitude for each neighborhood
dmadrid_merged = dmadrid_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

dmadrid_merged.head() 

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Palacio,40.415129,-3.715618,1.0,Plaza,Historic Site,Beer Garden,Tapas Restaurant,Church,Museum,Wine Shop,Wine Bar,Dessert Shop,Furniture / Home Store
1,Embajadores,40.409681,-3.701644,1.0,Pizza Place,Bar,Theater,Restaurant,Café,Tapas Restaurant,Coffee Shop,Pub,Record Shop,Souvlaki Shop
2,Cortes,40.414779,-3.697584,0.0,Spanish Restaurant,Hotel,Restaurant,Bar,Plaza,Japanese Restaurant,Tapas Restaurant,Nightclub,Cuban Restaurant,Chinese Restaurant
3,Justicia,40.423957,-3.695747,1.0,Bakery,Restaurant,Deli / Bodega,Spanish Restaurant,Vegetarian / Vegan Restaurant,Flower Shop,Hotel,Nightclub,Gay Bar,Plaza
4,Universidad,40.425409,-3.705989,1.0,Bar,Tapas Restaurant,Bookstore,Cocktail Bar,Gastropub,Pizza Place,Restaurant,Breakfast Spot,Mexican Restaurant,Spanish Restaurant


In [84]:
dmadrid_merged.to_excel("MadridMerged.xlsx")

In [88]:
df1 = pd.read_excel ("MadridMerged1.xlsx")
#df2 = df1.drop(['Unnamed: 0'], axis=1)
#df2

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Palacio,40.415129,-3.715618,1,Plaza,Historic Site,Beer Garden,Tapas Restaurant,Church,Museum,Wine Shop,Wine Bar,Dessert Shop,Furniture / Home Store
1,Embajadores,40.409681,-3.701644,1,Pizza Place,Bar,Theater,Restaurant,Café,Tapas Restaurant,Coffee Shop,Pub,Record Shop,Souvlaki Shop
2,Cortes,40.414779,-3.697584,0,Spanish Restaurant,Hotel,Restaurant,Bar,Plaza,Japanese Restaurant,Tapas Restaurant,Nightclub,Cuban Restaurant,Chinese Restaurant
3,Justicia,40.423957,-3.695747,1,Bakery,Restaurant,Deli / Bodega,Spanish Restaurant,Vegetarian / Vegan Restaurant,Flower Shop,Hotel,Nightclub,Gay Bar,Plaza
4,Universidad,40.425409,-3.705989,1,Bar,Tapas Restaurant,Bookstore,Cocktail Bar,Gastropub,Pizza Place,Restaurant,Breakfast Spot,Mexican Restaurant,Spanish Restaurant
5,Sol,40.417827,-3.702222,1,Tapas Restaurant,Hotel,Plaza,Hostel,Frozen Yogurt Shop,Seafood Restaurant,Peruvian Restaurant,Burger Joint,Café,Casino
6,Imperial,40.405800,-3.718685,1,Spa,Gym / Fitness Center,Pizza Place,Café,Tapas Restaurant,Chinese Restaurant,Gym,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
7,Acacias,40.404075,-3.705957,1,Gym / Fitness Center,Supermarket,Tapas Restaurant,Café,Spanish Restaurant,Indie Movie Theater,Sushi Restaurant,Bookstore,Bar,Gym
8,Chopera,40.394893,-3.699705,1,Grocery Store,Italian Restaurant,Bar,Plaza,Park,Café,Garden,Burger Joint,Spanish Restaurant,Tapas Restaurant
9,Legazpi,40.391172,-3.695190,1,Argentinian Restaurant,Bar,Flea Market,Bakery,Café,Seafood Restaurant,Spanish Restaurant,Coffee Shop,Grocery Store,Restaurant


Finally, let's visualize the resulting clusters

In [90]:
address = 'Madrid, Spain'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df2['Latitude'], df2['Longitude'], df2['Neighborhood'], df2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Madrid Clusters

In [92]:
df1.loc[df1['Cluster Labels'] == 0, df1.columns[[1] + list(range(5, df1.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Cortes,Spanish Restaurant,Hotel,Restaurant,Bar,Plaza,Japanese Restaurant,Tapas Restaurant,Nightclub,Cuban Restaurant,Chinese Restaurant
16,Ibiza,Spanish Restaurant,Restaurant,Brewery,Tapas Restaurant,Seafood Restaurant,Italian Restaurant,Bar,Mediterranean Restaurant,Outdoors & Recreation,Coffee Shop
18,Niño Jesús,Spanish Restaurant,Plaza,Athletics & Sports,Mexican Restaurant,Flower Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
19,Recoletos,Restaurant,Spanish Restaurant,Italian Restaurant,Clothing Store,Jewelry Store,Coffee Shop,Cosmetics Shop,Shoe Store,Accessories Store,Burger Joint
23,Castellana,Spanish Restaurant,Restaurant,Bistro,Coffee Shop,Japanese Restaurant,Bakery,Gastropub,Church,Food & Drink Shop,Sandwich Place
28,Nueva España,Spanish Restaurant,Big Box Store,Fast Food Restaurant,Gym,Food,Gas Station,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
30,Bellas Vistas,Spanish Restaurant,Bar,Seafood Restaurant,Women's Store,Flower Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
33,Valdeacederas,Breakfast Spot,Hobby Shop,Spanish Restaurant,Flower Shop,Gas Station,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
34,Berruguete,Spanish Restaurant,Bakery,Supermarket,Gym,Gym / Fitness Center,Deli / Bodega,Ice Cream Shop,IT Services,Fast Food Restaurant,Fried Chicken Joint
36,Arapiles,Bar,Spanish Restaurant,Theater,Restaurant,Sushi Restaurant,Café,Peruvian Restaurant,Coffee Shop,Brewery,Sandwich Place


In [93]:
df1.loc[df1['Cluster Labels'] == 1, df1.columns[[1] + list(range(5, df1.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Palacio,Plaza,Historic Site,Beer Garden,Tapas Restaurant,Church,Museum,Wine Shop,Wine Bar,Dessert Shop,Furniture / Home Store
1,Embajadores,Pizza Place,Bar,Theater,Restaurant,Café,Tapas Restaurant,Coffee Shop,Pub,Record Shop,Souvlaki Shop
3,Justicia,Bakery,Restaurant,Deli / Bodega,Spanish Restaurant,Vegetarian / Vegan Restaurant,Flower Shop,Hotel,Nightclub,Gay Bar,Plaza
4,Universidad,Bar,Tapas Restaurant,Bookstore,Cocktail Bar,Gastropub,Pizza Place,Restaurant,Breakfast Spot,Mexican Restaurant,Spanish Restaurant
5,Sol,Tapas Restaurant,Hotel,Plaza,Hostel,Frozen Yogurt Shop,Seafood Restaurant,Peruvian Restaurant,Burger Joint,Café,Casino
6,Imperial,Spa,Gym / Fitness Center,Pizza Place,Café,Tapas Restaurant,Chinese Restaurant,Gym,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
7,Acacias,Gym / Fitness Center,Supermarket,Tapas Restaurant,Café,Spanish Restaurant,Indie Movie Theater,Sushi Restaurant,Bookstore,Bar,Gym
8,Chopera,Grocery Store,Italian Restaurant,Bar,Plaza,Park,Café,Garden,Burger Joint,Spanish Restaurant,Tapas Restaurant
9,Legazpi,Argentinian Restaurant,Bar,Flea Market,Bakery,Café,Seafood Restaurant,Spanish Restaurant,Coffee Shop,Grocery Store,Restaurant
10,Delicias,Playground,Restaurant,Women's Store,Flea Market,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Fountain,Food Truck


In [94]:
df1.loc[df1['Cluster Labels'] == 2, df1.columns[[1] + list(range(5, df1.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,Fuentelarreina,Soccer Field,Tea Room,Women's Store,Gas Station,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Fountain,Food Truck
101,Santa Eugenia,Soccer Field,BBQ Joint,Women's Store,Flower Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Fountain
106,Arcos,Soccer Field,Women's Store,Flower Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Fountain,Food Truck


In [95]:
df1.loc[df1['Cluster Labels'] == 3, df1.columns[[1] + list(range(5, df1.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,Buenavista,BBQ Joint,Flea Market,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Fountain,Food Truck,Food & Drink Shop


In [96]:
df1.loc[df1['Cluster Labels'] == 4, df1.columns[[1] + list(range(5, df1.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
97,San Cristóbal,Insurance Office,Women's Store,Flea Market,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Fountain,Food Truck


## Explore Neighborhoods in Barcelona

##### Code to run the above function on each neighborhood and create a new dataframe called *dbarcelona_venues*.

In [97]:
dbarcelona_venues = getNearbyVenues(names=df_bcnll['Neighborhood'],
                                   latitudes=df_bcnll['Latitude'],
                                   longitudes=df_bcnll['Longitude']
                                  )


print('There are {} uniques categories.'.format(len(dmadrid_venues['Venue Category'].unique())))

El Raval
Barrio Gótico
La Barceloneta
Santa Caterina i la Ribera
El Fort Pienc
Sagrada Familia
Dreta de l''Eixample
Sant Antoni
La Marina del Prat Vermell
La Marina de Port
La Font de la Guatlla
Hostafrancs
La Bordeta
Sants-Badal
Sants
Zona Franca
Montjuic
Les Corts
La Maternitat i Sant Ramon
Pedralbes
Vallvidrera
el Tibidabo
les Planes
Sarrià
Las Tres Torres
Sant Gervasi-Bonanova
Sant Gervasi-Galvany
El Putget i Farró
El Coll
La Salud
Camp d'en Grassot i Gràcia Nova
Baix Guinardó
Can Baró
El Guinardó
La Font d'en Fargues
El Carmel
La Teixonera
Sant Genís dels Agudells
Montbau
La Clota
Horta
Vilapicina y La Torre Llobeta
Porta
Turó de la Peira
Can Peguera
La Guineueta
Canyelles
Les Roquetes
Verdún
La Prosperitat
La Trinitat Nova
Torre Baró
Ciudad Meridiana
Vallbona
La Trinitat Vella
Baró de Viver
El Buen Pastor
San Andrés
La Sagrera
Congrés i els Indians
Navas
El Clot
El Parc i la Llacuna del Poblenou
La Vila Olímpica del Poblenou
Diagonal Mar i Front Marítim del Poblenou
El Besós y el

#### Analyze Each Neighborhood

In [98]:
# one hot encoding
dbarcelona_onehot = pd.get_dummies(dbarcelona_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dbarcelona_onehot['Neighborhood'] = dbarcelona_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dbarcelona_onehot.columns[-1]] + list(dbarcelona_onehot.columns[:-1])
dbarcelona_onehot = dbarcelona_onehot[fixed_columns]

dbarcelona_onehot.head()


,Yoga Studio,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Baby Store,Bakery,Bar,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Burger Joint,Bus Station,Bus Stop,Cafeteria,Café,Candy Store,Casino,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,College Residence Hall,Comedy Club,Comic Shop,Convenience Store,Cosmetics Shop,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Electronics Store,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Travel,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,Historic Site,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Market,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mountain,Multiplex,Museum,Music Venue,National Park,Neighborhood,Nightclub,Optical Shop,Organic Grocery,Outdoor Sculpture,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Perfume Shop,Peruvian Restaurant,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Portuguese Restaurant,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Restaurant,Rock Club,Roof Deck,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,El Raval,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,El Raval,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,El Raval,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,El Raval,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

#### Group by neighborhood

In [99]:
dbarcelona_grouped = dbarcelona_onehot.groupby('Neighborhood').mean().reset_index()
dbarcelona_grouped

,Neighborhood,Yoga Studio,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Baby Store,Bakery,Bar,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Burger Joint,Bus Station,Bus Stop,Cafeteria,Café,Candy Store,Casino,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,College Residence Hall,Comedy Club,Comic Shop,Convenience Store,Cosmetics Shop,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Electronics Store,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Travel,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,Historic Site,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Market,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mountain,Multiplex,Museum,Music Venue,National Park,Nightclub,Optical Shop,Organic Grocery,Outdoor Sculpture,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Perfume Shop,Peruvian Restaurant,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Portuguese Restaurant,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Restaurant,Rock Club,Roof Deck,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Wine Shop
0,Baix Guinardó,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.083333,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.083333,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
1,Barrio Gótico,0.000000,0.000000,0.000000,0.00000

##### Let's print each neighborhood along with the top 5 most common venues

In [100]:
num_top_venues = 5

for hood in dbarcelona_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dbarcelona_grouped[dbarcelona_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')



----Baix Guinardó----
                       venue  freq
0           Tapas Restaurant  0.17
1              Grocery Store  0.08
2                       Café  0.08
3                Beer Garden  0.08
4         Salon / Barbershop  0.08
5                Supermarket  0.08
6  Latin American Restaurant  0.08
7                       Park  0.08
8             Cosmetics Shop  0.08
9                        Gym  0.08


----Barrio Gótico----
                           venue  freq
0                          Plaza  0.13
1             Spanish Restaurant  0.11
2                          Hotel  0.04
3                       Tea Room  0.04
4                       Wine Bar  0.04
5                   Dessert Shop  0.04
6  Vegetarian / Vegan Restaurant  0.04
7                 Ice Cream Shop  0.04
8                    Candy Store  0.04
9               Tapas Restaurant  0.04


----Camp d'en Grassot i Gràcia Nova----
                      venue  freq
0                    Bakery  0.12
1          Tapas Restaurant  0

##### Let's put it into a *pandas* dataframe

In [101]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dbarcelona_grouped['Neighborhood']

for ind in np.arange(dbarcelona_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dbarcelona_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Baix Guinardó,Tapas Restaurant,Gym,Grocery Store,Park,Plaza,Beer Garden,Cosmetics Shop,Latin American Restaurant,Salon / Barbershop,Café
1,Barrio Gótico,Plaza,Spanish Restaurant,Tea Room,Vegetarian / Vegan Restaurant,Ice Cream Shop,Dessert Shop,Candy Store,Hotel,Tapas Restaurant,Wine Bar
2,Camp d'en Grassot i Gràcia Nova,Bar,Bakery,Tapas Restaurant,Spanish Restaurant,Grocery Store,Dance Studio,Farmers Market,Café,Japanese Restaurant,Fruit & Vegetable Store
3,Can Baró,Grocery Store,Plaza,Chinese Restaurant,Dessert Shop,Wine Shop,Deli / Bodega,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop
4,Can Peguera,Playground,Café,Flower Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop,Exhibit,Electronics Store,Diner


###  Cluster Barcelona Neighborhoods

In [102]:
# set number of clusters
kclusters = 5

dbarcelona_grouped_clustering = dbarcelona_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dbarcelona_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0])

In [103]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dbarcelona_merged = df_bcnll

# merge barcelona_grouped with barcelona_data to add latitude/longitude for each neighborhood
dbarcelona_merged = dbarcelona_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

dbarcelona_merged.head() 

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,El Raval,41.379518,2.168368,0.0,Spanish Restaurant,Cocktail Bar,Mediterranean Restaurant,Tapas Restaurant,Indian Restaurant,Bar,Hotel,Dance Studio,Gastropub,Indie Theater
1,Barrio Gótico,41.383395,2.176912,0.0,Plaza,Spanish Restaurant,Tea Room,Vegetarian / Vegan Restaurant,Ice Cream Shop,Dessert Shop,Candy Store,Hotel,Tapas Restaurant,Wine Bar
2,La Barceloneta,41.380653,2.189927,0.0,Tapas Restaurant,Spanish Restaurant,Mediterranean Restaurant,Bar,Paella Restaurant,Bakery,Plaza,Seafood Restaurant,Wine Shop,Food
3,Santa Caterina i la Ribera,41.386650,2.184194,0.0,Tapas Restaurant,Hotel,Bar,Mexican Restaurant,Italian Restaurant,Gift Shop,Exhibit,Seafood Restaurant,Flea Market,Breakfast Spot
4,El Fort Pienc,41.395925,2.182325,0.0,Health Food Store,Gym,Soccer Stadium,Bistro,Coffee Shop,Grocery Store,Flea Market,Farmers Market,Falafel Restaurant,Fabric Shop


In [104]:
dbarcelona_merged.to_excel("BarcelonaMerged.xlsx")

In [105]:
df2 = pd.read_excel ("BarcelonaMerged1.xlsx")

Finally, let's visualize the resulting clusters

In [106]:
address = 'Barcelona, Spain'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df2['Latitude'], df2['Longitude'], df2['Neighborhood'], df2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Barcelona Clusters

In [107]:
df2.loc[df2['Cluster Labels'] == 0, df2.columns[[1] + list(range(5, df2.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,El Raval,Spanish Restaurant,Cocktail Bar,Mediterranean Restaurant,Tapas Restaurant,Indian Restaurant,Bar,Hotel,Dance Studio,Gastropub,Indie Theater
1,Barrio Gótico,Plaza,Spanish Restaurant,Tea Room,Vegetarian / Vegan Restaurant,Ice Cream Shop,Dessert Shop,Candy Store,Hotel,Tapas Restaurant,Wine Bar
2,La Barceloneta,Tapas Restaurant,Spanish Restaurant,Mediterranean Restaurant,Bar,Paella Restaurant,Bakery,Plaza,Seafood Restaurant,Wine Shop,Food
3,Santa Caterina i la Ribera,Tapas Restaurant,Hotel,Bar,Mexican Restaurant,Italian Restaurant,Gift Shop,Exhibit,Seafood Restaurant,Flea Market,Breakfast Spot
4,El Fort Pienc,Health Food Store,Gym,Soccer Stadium,Bistro,Coffee Shop,Grocery Store,Flea Market,Farmers Market,Falafel Restaurant,Fabric Shop
5,Sagrada Familia,Coffee Shop,Mexican Restaurant,Italian Restaurant,Restaurant,Café,Burger Joint,Wine Bar,Historic Site,Pizza Place,Plaza
6,Dreta de l''Eixample,Hotel,Hostel,Bookstore,Beer Bar,Tapas Restaurant,Boutique,Spa,Sporting Goods Shop,Roof Deck,Café
7,Sant Antoni,Café,Bar,Mediterranean Restaurant,Coffee Shop,Spanish Restaurant,Pizza Place,Tapas Restaurant,Restaurant,Shop & Service,Italian Restaurant
8,La Marina del Prat Vermell,Mediterranean Restaurant,Restaurant,Plaza,Bus Station,Gastropub,Spanish Restaurant,Bus Stop,Bakery,Metro Station,Supermarket
9,La Marina de Port,Grocery Store,Café,Farmers Market,Spanish Restaurant,Sporting Goods Shop,Diner,Bakery,Supermarket,Dessert Shop,Fast Food Restaurant


In [108]:
df2.loc[df2['Cluster Labels'] == 1, df2.columns[[1] + list(range(5, df2.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,Horta,Hostel,Clothing Store,Sports Club,Café,Bakery,Gym / Fitness Center,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant
38,Can Peguera,Playground,Café,Flower Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop,Exhibit,Electronics Store,Diner
49,El Buen Pastor,Café,Metro Station,Paper / Office Supplies Store,Restaurant,Mediterranean Restaurant,Convenience Store,Cosmetics Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
60,La Verneda y la Paz,Café,Tapas Restaurant,Pharmacy,College Residence Hall,Comedy Club,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop,Exhibit


In [109]:
df2.loc[df2['Cluster Labels'] == 2, df2.columns[[1] + list(range(5, df2.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,La Bordeta,Restaurant,Coffee Shop,Plaza,Theater,Supermarket,Bakery,Dance Studio,Falafel Restaurant,Fabric Shop,Exhibit
25,La Salud,Restaurant,Plaza,Bakery,Wine Shop,Dance Studio,Farmers Market,Falafel Restaurant,Fabric Shop,Exhibit,Electronics Store
30,El Carmel,Coffee Shop,Plaza,Grocery Store,Park,Diner,Supermarket,Deli / Bodega,Farmers Market,Falafel Restaurant,Fabric Shop
36,Porta,Restaurant,Plaza,Public Art,Sandwich Place,Cupcake Shop,Fabric Shop,Exhibit,Electronics Store,Diner,Dessert Shop


In [110]:
df2.loc[df2['Cluster Labels'] == 3, df2.columns[[1] + list(range(5, df2.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,Torre Baró,Optical Shop,Wine Shop,College Cafeteria,Flea Market,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop,Exhibit,Electronics Store


In [111]:
df2.loc[df2['Cluster Labels'] == 4, df2.columns[[1] + list(range(5, df2.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,La Clota,Garden Center,Wine Shop,Flower Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop,Exhibit,Electronics Store,Diner
